## Minimal Recurrent Neural Network in Julia
This is an implementation of the simplest RNN model presented in Andrei Karpathy's awesome article, ["The Unreasonable Effectiveness of Recurrent Neural Networks"](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), written in [Julia](http://julialang.org/).

Karpathy presents a 100-line ["minimal recurrent nerual network"](https://gist.github.com/karpathy/d4dee566867f8291f086), written in Python. This notebook reimplements the model and training code, aiming less to be short and more to be clear and instructive.

The network will be trained with a body of text, character by character, and learn to predict the pattern of that text. We will use it to generate, or "hallucinate," new text that *feels* like the original!


## The Data

In this case my data is all of Shakespeare's commedies concatenated in a single file, but any large text file will do!

In [1]:
original_text = ""
open("all_comedies_cat.txt") do original_text_file
    global original_text
    original_text = readall(original_text_file)
end
# Print some samples from the text, just to get a feel for it.
println("Length: ", length(original_text))
println("\n --------- Some Sample Text: -------- \n")
println(original_text[1:200])
println("\n --------- ... ---------------------- \n")
print(original_text[end-1000:end])

Length: 2108622

 --------- Some Sample Text: -------- 

	ALL'S WELL THAT ENDS WELL


	DRAMATIS PERSONAE


KING OF FRANCE	(KING:)

DUKE OF FLORENCE	(DUKE:)

BERTRAM	Count of Rousillon.

LAFEU	an old lord.

PAROLLES	a follower of Bertram.


Steward	|
	|  ser

 --------- ... ---------------------- 

y mate, that's never to be found again,
	Lament till I am lost.

LEONTES	O, peace, Paulina!
	Thou shouldst a husband take by my consent,
	As I by thine a wife: this is a match,
	And made between's by vows. Thou hast found mine;
	But how, is to be question'd; for I saw her,
	As I thought, dead, and have in vain said many
	A prayer upon her grave. I'll not seek far--
	For him, I partly know his mind--to find thee
	An honourable husband. Come, Camillo,
	And take her by the hand, whose worth and honesty
	Is richly noted and here justified
	By us, a pair of kings. Let's from this place.
	What! look upon my brother: both your pardons,
	That e'er I put between your holy looks
	My ill suspicion. This

In [2]:
alphabet = sort(unique(original_text))  # Converts an index into a character.
println(length(alphabet))

reverse_alphabet = [ch => i for (i,ch) in enumerate(alphabet)]  # Converts a char into an index.

69


Dict{Any,Int64} with 69 entries:
  'D'  => 18
  '|'  => 69
  'Y'  => 39
  '\'' => 6
  '.'  => 11
  'U'  => 35
  'B'  => 16
  ':'  => 12
  ';'  => 13
  'J'  => 24
  'Z'  => 40
  'o'  => 57
  'N'  => 28
  'p'  => 58
  'F'  => 20
  'j'  => 52
  '!'  => 4
  'y'  => 67
  'E'  => 19
  'r'  => 60
  'm'  => 55
  'S'  => 33
  'A'  => 15
  ','  => 9
  'T'  => 34
  ⋮    => ⋮

----------
## Define the Model

This is modelled after Karpathy's minimal RNN, described in his [excellent article](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and implemented in this [short gist](https://gist.github.com/karpathy/d4dee566867f8291f086). From the article, the basic step function is:

```
# update the hidden state
self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
# compute the output vector
y = np.dot(self.W_hy, self.h)
```

There are also two biases, `bh` and `by`, used to compute self.h and y. So the model has 5 learned parameters: `Wxh`, `Whh`, `Why`, `bh`, and `by`.

Mathematically, each step looks like

$$
h' = tanh(W_{hh} * h + W_{xh} * x) + b_h
$$
$$
y = W_{hy} * h' + b_y
$$

where $x$ is the input for the step (a vector that represents a character), $h$ is the hidden state left from the previous step, and $y$ is the predicted output vector (a probability distribution over the set of characters).

Each of these vectors represent the "neurons" in the neural network; each cell a neuron. $x$ holds the input neurons, $h$ the hidden neurons, and $y$ the output neurons. The connections between these neurons are represented by the weight matrices and bias vectors. For example, the cells in $W_{hx}$ control how heavily each input neuron from $x$ should affect the new value for each hidden neuron in $h$. The weights and biases -- the parameters -- are learned from training the network with successive batches of inputs.

### Representing the Network

We will represent this network using a Julia type we'll call `RNN`, holding all the parameters (weights+biases) and the hidden state vector (hidden neuron layer). Note that the hidden state is not trained like the parameters, but can is state held by the network during operation. Therefore, we will still store it inside the class.

In [3]:
"""
A Recurrent Neural Network is a collection of weight matrices, which are updated after each training batch,
and a "hidden state", h, which is updated after each classification in the batch and carried throughout the whole batch.
"""
type RNN
    # The Weights and Biases (matrices)
    Wxh   # Controls how each input neuron affects the new hidden neuron values.
    Whh   # Controls how each hidden neuron affects the new hidden neuron values.
    Why   # Controls how to calculate the output neurons from the hidden layer.
    bh    # Bias added to new hidden state.
    by    # Bias added to calculated output vector.

    # Adagrad update "memory" weights. This is an implementation detail of the Adagrad update algorithm.
    mWxh
    mWhh
    mWhy
    mbh
    mby
    
    # The hidden state is updated by each step, and is a part of generating the next step's classification. It is not
    # updated by backpropogation (training), but it's part of calculating the new update for the weights after each step.
    # Updating the hidden state and carrying it to the next step is called "unrolling" the RNN, and is what makes an
    # RNN "deep", similar to having many hidden layers in a more traditional RNN.
    h     # Not a variable, but still associated with the RNN.
    
    function RNN(in_dim::Integer, hidden_dim::Integer, out_dim::Integer)
        # Initialize the Weights/Biases variables
        Wxh = randn(hidden_dim, in_dim)*0.01      # scaled way down.
        Whh = randn(hidden_dim, hidden_dim)*0.01  # Initialize these with random values from normal distribution,
        Why = randn(out_dim, hidden_dim)*0.01
        bh = zeros(hidden_dim, 1)
        by = zeros(out_dim, 1)
        
        # Initialize the memory variables
        mWxh, mWhh, mWhy, mbh, mby = zeros(Wxh), zeros(Whh), zeros(Why), zeros(bh), zeros(by)

        # Initialize the hidden state vector.
        h = zeros(bh)
        
        new(Wxh, Whh, Why, bh, by, mWxh, mWhh, mWhy, mbh, mby, h)
    end
end
# Test code
r = RNN(1,2,1)
println(r.Wxh)
println(r.h)

[0.0017844250981475835
 0.008640373687761155]
[0.0
 0.0]


### Setting up the recurrence

A recurrent neural net is *recurrent* because the simple model is repeated multiple times to create the overall model, carrying state from each step to the next. This is called "unrolling" the net.

This matters, because in order for backpropogation to calculate the effect that the hidden state has on the loss, it needs to take the changes from one `state` to another into account when calculating the gradient. If you didn't unroll the network, and instead just returned the `state` from each run and passed the `state` back into the next run, the `state`'s only effect on the cost would be how the `state` effected `y`, not how the current `state` effected the next state.

For example, the simple network without unrolling it looks like this:
  
$  cost = truth - y $, where $ y = W_{hy}*h' $ and $h' = tanh(W_{hh}*h + W_{xh}*x) $

The current state, $h$, only effects $cost$ through it's impact on $y$. Even though it sets the *next state*, $h'$, the transition from $h$ to $h'$ is never considered during backpropogation.

Instead, an *unrolled* network *does* effect the cost both from the current $h$ and on its effect on the next $h$. Consider a network unrolled for 2 steps, $x_0$ and $x_1$:

$  cost = (truth_1 - y_1) + (truth_0 - y_0) $

for $ y_1 = W_{hy}*h_2 $, $h_2 = tanh(W_{hh}*h_1 + W_{xh}*x_1) $

and
$ y_0 = W_{hy}*h_1 $, $h_1 = tanh(W_{hh}*h_0 + W_{xh}*x_0) $

Now, $W_{hh}$'s impact on $h_1$ is reflected in the `cost`, alongside its impact on $y_0$.

SO, I *think*, the more steps you unroll a network for before doing backpropogation, the more emphasis you're placing on the hidden state's impact versus the other weights.

In [4]:
"""
A helper function to prevent spiraling values when doing derivatives.
"""
function clip_values(deltas_matrix, max_threshold::Float64, min_threshold::Float64)
    deltas_matrix = max(deltas_matrix, min_threshold)
    deltas_matrix = min(deltas_matrix, max_threshold)
    return deltas_matrix
end
# Test code
clip_values([1.0 -11.0 6.0]', 5.0, -5.0)

3x1 Array{Float64,2}:
  1.0
 -5.0
  5.0

In [5]:
"""
    forward_pass_step(r::RNN, in_h, x)

A single step in the RNN: calculate a new output vector and new hidden
state from an input vector (a one-hot vector) and the current hidden state.

Returns the output vector as a raw vector and as a normalized
vector, which can be thought of as a probability distribution, and the new hidden state.
"""
function forward_pass_step(r::RNN, in_h, x)
    const local Wxh = r.Wxh
    const local Whh = r.Whh
    const local Why = r.Why
    const local bh = r.bh
    const local by = r.by

    # Calculate the new h.
    const new_h = tanh(Wxh*x + Whh*in_h + bh)

    const y = Why*new_h + by
    
    # --- Normalize y so that it represents a probability distribution over the set of characters.
    
    # Taking exp(y) conveniently eliminates negative values from tanh.
    const exp_y = exp(y)     # 2x1 (pointwise)
    const y_norm = exp_y ./ sum(exp(y)) # 2x1 Element wise division 

    return y_norm, y, new_h
end

# Test code
r1 = RNN(2,4,3)
r1.h = [0 0 0 0.]'
y1, _, r1.h = forward_pass_step(r1, r1.h, [1.0 0]')
y2, _, r1.h = forward_pass_step(r1, r1.h, [0.0 0.1]')
y3, _, r1.h = forward_pass_step(r1, r1.h, [1.0 0.]')
println("ys:\n$y1\n$y2\n$y3")
println("h:\n",r1.h)

ys:
[0.33330945985467153
 0.3333195712203447
 0.3333709689249837]
[0.3333325336240126
 0.3333333587436077
 0.3333341076323797]
[0.3333094638455846
 0.3333195728680036
 0.3333709632864119]
h:
[0.005225351549333576
 0.019163877595654712
 -0.002894982134721514
 0.00544056088368608]


In [7]:
"""
    forward_pass_backpropogate_batch(r::RNN, xs, ts)

Unroll the RNN for all the inputs in this batch, then perform backpropogation and calculate the
delta updates for its parameters. Returns the `cost` calculated over this batch, and the delta updates.

This function modifies r.h, and leaves it in its new state.

r - an RNN.
x - The batch of input vectors: a tuple of one-hot vectors.
t - The batch of "truth" vectors: a tuple of one-hot vectors of same count as xs.
"""
function forward_pass_backpropogate_batch(r::RNN, xs, ts)

    #Forward Pass
    hs, ys, ps = Dict(), Dict(), Dict()
    hs[0] = r.h
    local Wxh = r.Wxh
    local Whh = r.Whh
    local Why = r.Why
    local bh = r.bh
    local by = r.by

    out_delta_max_threshold = 5.0   # Tweakable.
    out_delta_min_threshold = -5.0   # Tweakable.

    cost = 0
    
    # Unroll the network.
    # For each input, perform the forward pass step and update the hidden state. Keep track
    # of the state after each step so we can calculate the derivatives correctly during backpropogation.
    for i in 1:length(ts)
        local h = hs[i-1]
        local x = xs[i]
        local t = ts[i]

        ps[i], ys[i], hs[i] = forward_pass_step(r, h, x)

        # Take the dot-product with t-inverse, to get only
        # the value from log(ps[i]) which corresponds to the truth.
        prediction_for_truth_value = t' * ps[i]  # 1x1 (scalar-ish)

        # The log-cost reflects how close the prediction for the truth value was to 1.
        cost += -log(prediction_for_truth_value)
    end

    # ----------  Now go back down:
    # Starting from the cost, calculate the derivative (gradient) of the cost, with respect to each variable,
    # or, *how much the cost changes if you change each variable*. This allows us to move each variable along
    # it's derivative to lower the cost for this batch.

    # These will be our final outputs, they represent ``δcost/δvariable``.
    dWxh = zeros(r.Wxh)
    dWhh = zeros(r.Whh)
    dWhy = zeros(r.Why)
    dbh  = zeros(r.bh)
    dby  = zeros(r.by)
    
    ∂cost_∂hnext = zeros(r.h)
    for i in length(ts):-1:1

        # Copied from Karpathy. I don't know what these two lines mean.
        #dy = ps[i]
        #dy -= ts[i] # backprop into y
        #dWhy += dy * hs[i]'
        #dby += dy
        #dh = Why' * dy + dhnext # backprop into h
        #dhraw = (1 - hs[i] .* hs[i]) .* dh # backprop through tanh nonlinearity
        #dbh += dhraw
        #dWxh += dhraw * xs[i]'
        #dWhh += dhraw * hs[i-1]'
        #dhnext = Whh' * dhraw
        
        
        ∂cost_∂y = ps[i]
        ∂cost_∂y -= ts[i] # backprop into y
 
        ∂cost_∂Why = ∂cost_∂y * hs[i]'  # 2x3
        ∂cost_∂by = ∂cost_∂y            # 2x1
 
        ∂cost_∂h = Why' * ∂cost_∂y  +  ∂cost_∂hnext   # 3x1
        dhraw = (1 - hs[i] .^ 2) .* ∂cost_∂h    # "backprop through tanh nonlinearity"
 
        ∂cost_∂bh = dhraw  # 3x1    # noop
 
        ∂cost_∂Whh = dhraw * hs[i-1]'   # 3x3
        ∂cost_∂Wxh = dhraw * xs[i]'   # 3x2
 
        ∂cost_∂hnext = Whh' * dhraw  # 3x1
 
        # Update the final derivatives
 
        dWxh += ∂cost_∂Wxh
        dWhh += ∂cost_∂Whh
        dWhy += ∂cost_∂Why
        dbh  += ∂cost_∂bh
        dby  += ∂cost_∂by


        #println("∂cost_∂Wxh:$∂cost_∂Wxh")
    end

    # Clip deltas to mitigate exploding gradients.
    dWxh = clip_values(dWxh, out_delta_max_threshold, out_delta_min_threshold) 
    dWhh = clip_values(dWhh, out_delta_max_threshold, out_delta_min_threshold)
    dWhy = clip_values(dWhy, out_delta_max_threshold, out_delta_min_threshold)
    dbh = clip_values(dbh, out_delta_max_threshold, out_delta_min_threshold)
    dby = clip_values(dby, out_delta_max_threshold, out_delta_min_threshold)
     
    # Update the hidden state from this run.
    r.h = hs[length(ts)]

    return cost, dWxh, dWhh, dWhy, dbh, dby 

end

r = RNN(2,3,2)
r.Wxh = [.5 .2 ; 0.1 0.1 ; 0.2 0.2]
r.Whh = [.1 .1 .1 ; .2 .2 .2 ; 0.3 0.3 .3]
r.Why = [.4 .5 .6; .7 .8 .9 ]
r.h = [0.4 .2 0.8]'
println(forward_pass_backpropogate_batch(r, ([1 0]', [0 1]', [0 1]'), ([0 1]', [1 0]', [0 1]')))
println("\n",r.h)

(
[1.9302244965501878],

[-0.04238982839396551 -0.0059616523702824
 -0.054066359497087255 -0.0074679389032205196
 -0.04334667562232223 -0.009150013099945634],

[0.0053131034170606425 -0.012153131298957148 -0.039003515265967854
 -0.0006733502928015604 -0.015007342671044354 -0.04914214346576764
 -0.001873647622807225 -0.01320572057049087 -0.04117375097267407],

[0.15236661289368483 0.05868962310872297 0.09448040464381108
 -0.15236661289368486 -0.058689623108723 -0.0944804046438111],

[-0.04835148076424791
 -0.061534298400307774
 -0.05249668872226786],

[0.20836354484521263
 -0.2083635448452127])

[0.3165562782891978
 0.34135939193526
 0.5251652648834423]


## Hyperparameters

The last part of the model is the most important part for expirementing! These are the settings for the RNN we will use to learn our text. These are *not* the `parameters`, or variables, learned inside the network, but rather the options for setting up the network.

- `hidden_size` -- How many neurons to have in the hidden layer.
- `seq_length` -- How many characters to include in a single training batch. This translates to *how many steps to unroll the network during training*, which, remember is the crucial detail that makes an RNN recurrent. The larger this is, the more data it will take to train, but it will possibly be able to learn features that are more spread out in time (such as opening and closing brackets, maybe). On the other hand, if this is tiny, it will get feedback for each character and can learn with far less data, but might not learn spelling/structure as well.
- `learning_rate` -- How much to adjust each parameter along the gradients, or derivatives, calculated from backpropogation. If this is too large, the model can swing wildly and never settle. We use an "Adagrad update" which slowly shrinks the `learning_rate` over time, so it can start a bit larger than one would do without using adagrad.

Play with these to see how the network changes!

In [6]:
# Hyperparamaters
hidden_size = 100           # In Karpathy's min-char-rnn.py, this is set to 100.
seq_length = 50             # In Karpathy's min-char-rnn.py, this is set to 25.
learning_rate = 1e-1        # In Karpathy's min-char-rnn.py, this is set to 1e-1.

0.1

Training!
---------
Now let's do the training!

Concept:

Calculate the gradient of the Cost with respect to each of the weights, i.e. the partial derivates $\partial$Cost / $\partial$Weight for each weight.

This gradient represents how much the Cost would change if we update the weights a tiny bit. So we subtract the gradient from the weight to decrease the Cost!

Note: We don't even care what $y$ is, we only use it to calculate the gradient for a given set of weights. So that information can stay in the backpropogate function.

In [8]:
"""
    update(r, xs, ts)

Train on a batch of inputs and truths, and update the model.

Performs Adagrad update gradient descent using the gradients calculated from backpropogation.

r - RNN
xs - a batch of inputs
ts - a batch of truths (as one-hot nx1 matrices)
"""
function update(r, xs, ts)
    loss, ∂cost_∂Wxh, ∂cost_∂Whh, ∂cost_∂Why, ∂cost_∂bh, ∂cost_∂by = forward_pass_backpropogate_batch(r,xs,ts)
    
    # Adagrad update (Gradient Descent)
    # As these terms grow, the shift below becomes smaller, because they're the denominator.
    r.mWxh += ∂cost_∂Wxh .^ 2
    r.mWhh += ∂cost_∂Whh .^ 2
    r.mWhy += ∂cost_∂Why .^ 2
    r.mbh += ∂cost_∂bh .^ 2
    r.mby += ∂cost_∂by .^ 2
    
    # Shift the weights down along their gradients (derivates).
    # Remember that ∂cost_∂Wxh specifies how much cost will *increase* with an
    # increase in Wxh, so to decrease cost, you would move along negative ∂cost_∂Wxh.
    r.Wxh -= learning_rate * ∂cost_∂Wxh  ./ sqrt(r.mWxh + 1e-8)
    r.Whh -= learning_rate * ∂cost_∂Whh  ./ sqrt(r.mWhh + 1e-8)
    r.Why -= learning_rate * ∂cost_∂Why  ./ sqrt(r.mWhy + 1e-8)
    r.bh -= learning_rate * ∂cost_∂bh    ./ sqrt(r.mbh + 1e-8)
    r.by -= learning_rate * ∂cost_∂by    ./ sqrt(r.mby + 1e-8)
    
    return loss
end

# Test Code:
r = RNN(2,3,2)
r.Wxh = [.5 .2 ; 0.1 0.1 ; 0.2 0.2]
r.Whh = [.1 .1 .1 ; .2 .2 .2 ; 0.3 0.3 .3]
r.Why = [.4 .5 .6; .7 .8 .9 ]
r.h = [0.4 .2 0.8]'
loss = update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))
println(loss, r.h)

update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))
update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))
loss = update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))

println(loss, r.h)

println(r.Wxh, r.Whh, r.Why, r.bh, r.by)

[2.8513927492820152][0.5499489269489144
 0.32445312710513785
 0.5042017549258068]
[2.7732563220837685][0.2733504302372906
 -0.1218558122595025
 -0.03313777320467589]
[0.3673099058722714 0.10641030522686429
 -0.026636732258434413 0.006942351574688553
 0.06798023469772287 0.11654742554190307][0.014194231453489218 0.02240190796108634 0.015019502381031684
 0.11765274426740126 0.12166334880633656 0.11498598332296982
 0.22201671040103732 0.22457719522088102 0.2173526964776151][0.4753469640869792 0.5827236545826828 0.6514414354375764
 0.6246530359130209 0.7172763454173171 0.8485585645624236][-0.11227657394104021
 -0.11147721367622182
 -0.11101694366027448][0.021547450149868995
 -0.021547450149868946]


In [9]:
"""
    make_one_hot(length, index)

A helper function to make a "one-hot" vector, or a vector which is
all zeros with a one at the specified index. We will use this to
represent a character as input to the neural network.
"""
function make_one_hot(length, index)
    v = zeros(Float64, length, 1)
    v[index] = 1.0
    return v
end
x = make_one_hot(length(alphabet), reverse_alphabet['.'])
assert(1 == x[reverse_alphabet['.']])
assert(0 == x[reverse_alphabet['a']])

In [10]:
# Because Julia doesn't seem to have a function to sample from a set of probabilities?

"""
    rand_uniform(a, b)

Returns a number from the uniform random distribution between a and b.
"""
function rand_uniform(a, b)
    a + rand()*(b - a)
end
"""
    SampleFrom(probabilities)

Returns an index between 0 and length(probabilities), chosen based on the provided stepwise probabilities.
"""
function SampleFrom(probabilities)

    # Sum to create CDF:
    cdf = Array(Float64, 0)
    sum = 0.0
    for p in probabilities
        push!(cdf, sum + p)
        sum = cdf[end];
    end
        
    # Choose from CDF:
    cdf_value = rand_uniform(0.0,cdf[end])
    index = searchsortedfirst(cdf, cdf_value);

    return index;

end
# Test Code: run this repeatedly to see how the randomness follows the given distributions.
println(SampleFrom([1 1 1 1]))
println(SampleFrom([1 1 10 10]))
println(SampleFrom([0.1 0.2 0.7 0.8]))
#println(hist([SampleFrom([1 1 10 10]) for i in 0:1000]))

4
4
2


In [11]:
"""
    hallucinate(r, seed_idx, num_chars)

Generates, "hallucinates," text from the given RNN by repeatedly
1. running the network with an input,
2. generating an output probability distribution,
3. randomly selecting a new index from that distribution,
4. and using that index as a new input.
It does this num_chars times, to generate text num_chars long.
"""
function hallucinate(r, seed_idx, num_chars)
    hallucination = ""
    prev_ids = [seed_idx]
    # Should we clear the hidden state (not the weights)?
    # Note that Karpathy doesn't modify h, but I think it
    # makes sense to start over before each hallucination...
    # I don't do it here, though, to keep with his implementation.
    for x in range(1,num_chars)
        x_vec = make_one_hot(length(alphabet), prev_ids[end])
        y_norm,y,r.h = forward_pass_step(r, r.h, x_vec)

        # Now sample from y!
        letter_idx = SampleFrom(y_norm')
        #letter_idx = indmax(y)
        char = alphabet[letter_idx]
        
        append!(prev_ids,[letter_idx])
        hallucination = "$hallucination$char"
    end
    return hallucination
end
r = RNN(length(alphabet), 500, length(alphabet))
hallucinate(r, rand(1:length(alphabet)), 100)

"X;,xOnOqEdxtHRom,[.nbj[w,:o[qCW;FK!dIF&qzvyE]\n-rzBWclnZ]ykmSM[MP?:ses\nk)X|JhZuE|wV.qGZwKna,Kmd&U[\ntO"

----------

# Run it!

Alright! Let's run it! We set up a loop below which iterates through the text, one batch of `seq_length` characters at a time. For each batch, it runs `update`, which trains the RNN by iterating through the batch, backpropogating, and updating the weights using gradient descent. Each training step needs to operate on a batch, because unrolling the RNN for all the characters in the batch is what makes the Recurrent Neural Network *deep* -- and, as I understand it, being deep is what makes backpropogation/gradient descent somehow magically be a decently bowl-shaped space.

In [12]:
training_rnn = RNN(length(alphabet), hidden_size, length(alphabet))
chars_trained = 1

1

In [13]:
i = 0
smooth_loss = -log(1.0/length(alphabet))*seq_length # loss at iteration 0
println("======== Iteration: $i, Chars Trained:", chars_trained-1, "/", length(original_text), " Cost: $smooth_loss ===========")
while true
    if chars_trained+seq_length+1 >= length(original_text)
        println("========= DONE!! ===========")
        break
    end
    x_vecs = [make_one_hot(length(alphabet), reverse_alphabet[x]) for x in original_text[chars_trained:chars_trained+seq_length]] 
    truth_vecs = [make_one_hot(length(alphabet), reverse_alphabet[y_]) for y_ in original_text[chars_trained+1:chars_trained+seq_length+1]]

    if i % 100 == 0
        println("======== Iteration: $i, Chars Trained:", chars_trained-1, "/", length(original_text), " Cost: $smooth_loss ===========")
        seed_idx = reverse_alphabet[original_text[chars_trained]]
        println(hallucinate(training_rnn, seed_idx, 100))
    end

    (loss,) = update(training_rnn, x_vecs, truth_vecs)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    chars_trained += seq_length
    i += 1

end

======== Iteration: 0, Chars Trained:0/2108622 Cost: 211.70532522986298 ===========
======== Iteration: 0, Chars Trained:0/2108622 Cost: 211.70532522986298 ===========
ME[DAAE(q 
kf|t;FK)!nhb ?'oxVUHBkhl|Rphv-mI;ymIRQ!vr&qrWsFcmn[[|
kz.t	BSzv|w;jrJQIM'GPDejI;e:lb)xv.Q
======== Iteration: 100, Chars Trained:5000/2108622 Cost: 213.8570015065917 ===========
A  lalsn
LNfatsZkIf	srh strgsdPlElo, tNin st.T!to oo l gHd)tllUuBthekas hih al oht Hnrv,KZ	evtai;,ie
======== Iteration: 200, Chars Trained:10000/2108622 Cost: 209.00623347906495 ===========
eoy evaeelbowriwoe itp rss ctaun bel tyee antotht:y

PJRSAA	Tluhomgeig

GLAHSlJ T	mhdis,
	(fAi| wtt 
======== Iteration: 300, Chars Trained:15000/2108622 Cost: 203.6009864389416 ===========
kls se ma t'veu
	qr ooock t verenurymy
	Kluy cheidm mivuvcgand sheien tbuyf n

	ZIve|
!
NEjEREC&	ris
======== Iteration: 400, Chars Trained:20000/2108622 Cost: 197.56577771053566 ===========
uy

	wcos nop gowM,
	on goctetiud her,,
	mnung siard
	.hatnurelatef set

LoadError: LoadError: InterruptException:
while loading In[13], in expression starting on line 5

------------------------
Awesome, no!?

The cost eventually stops decreasing, because the Adagrad update is set to lower the learning rate over time. I can't remember why that's desirable. I *think* it's to prevent over-fitting. Anyway, I stopped the above run just about half-way through, because it seemed to have settled.


Let's generate a bunch of text now to get a feel for what it's learned:

In [28]:
println(hallucinate(training_rnn, reverse_alphabet['\n'], 5000))  # Start with an endline just so it's pretty.

	T andord revent my for fove your mabest not the Feltol-dert have P! She CASTIR	Wa sher pay: feavide? hackn.

S QUICK]


SHALUC	It and puspadr
	whave youl mat woud browt ver the, R II Hellided good' say atee, ous lidr gage, cabrot
	of thos
	hood!

PADUS	To wezredp?

Folllly shy willl Heristars, no sow your
	for, ap he wis haceddervoy, and wearct. Whot tof him bures! From's doud the reskn to
	tondlast! Thou my aind.

	[Anck masfing!

	[Enten bath to canig] nave wintor balak Sollas: Mive muth be toll onstol suegl ards hean?

DAGE]


	Ho TIST IOR]


	[Anter that mine pas, Shaplosion to'd and, rempoved wompe vard, jever you her youngh atlice
	of of whad 'ly in Shat! Caph rissittredy you, dostin-'s frin a pand com. I
	fo, tislest, shes chissy you pur.

dS CALUSS	I was meper.

Pgund I CE SHALOR]

SARDY	A Jeppodkivir.

PORDI	I hompland, Fary my he. Scomegor your im that, you.

	[Exchter kand the tiCk yould the she prove; he ames
	not,
	And did misear frecare,
	ponce cor is will we that you, d

----------------------------------

I like that it seems to be better at *starting* bracketed stage-instructions than closing them.

But it's amazing how quickly it learned the format of the text. Almost immediately, after just a few thousand characters, it learns to put the names in ALL CAPS and to indent the body of the text. In the above final output, it block-formats the stage-instructions.

This simple model performs very well on learning structure, it would seem.

## UNIT TESTS

From a modified version of Karpathy's python code:
```
# hyperparameters
hidden_size = 3 # size of hidden layer of neurons
vocab_size = 2

# model parameters
Wxh = np.array([[.5, .2], [0.1, 0.1], [0.2, 0.2]])
Whh = np.array([[.1, .1, .1], [.2, .2, .2], [.3, .3, .3]])
Why = np.array([[.4, .5, .6], [.7, .8, .9]])
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

h = [[0.4], [.2], [0.8]]

for x in lossFun([0,1], [1,0], h):
    print x, "\n"
```

Out:
```
1.39887501287 

[[-0.02348709  0.15845009]
 [-0.02995675  0.15314729]
 [-0.02401726  0.12098114]] 

[[ 0.08011355  0.05277361  0.06853661]
 [ 0.07453014  0.04955632  0.06043836]
 [ 0.05873529  0.03907731  0.04746229]] 

[[ 0.02188566 -0.0820149  -0.12198684]
 [-0.02188566  0.0820149   0.12198684]] 

[[ 0.13496299]
 [ 0.12319054]
 [ 0.09696389]] 

[[-0.20382526]
 [ 0.20382526]] 

[[ 0.33448831]
 [ 0.37630407]
 [ 0.56735968]] 
```

In [18]:
r = RNN(2,3,2)
r.Wxh = [.5 .2 ; 0.1 0.1 ; 0.2 0.2]
r.Whh = [.1 .1 .1 ; .2 .2 .2 ; 0.3 0.3 .3]
r.Why = [.4 .5 .6; .7 .8 .9 ]
r.h = [0.4 .2 0.8]'
outs = forward_pass_backpropogate_batch(r, ([1 0]', [0 1]'), ([0 1]', [1 0]'))
truth = (
[1.3988750128749587]',

[-0.02348709404610685 0.15845008784877856
 -0.029956754210863596 0.15314729364197688
 -0.02401725804279269 0.12098114381203691],

[0.08011354615577733 0.052773611291928826 0.06853660930090805
 0.07453013601361681 0.049556316201140885 0.06043836265216451
 0.058735290825127406 0.03907731268820138 0.04746229284518379],

[0.02188565806931217 -0.08201489754933375 -0.12198683957866582
 -0.021885658069312197 0.08201489754933372 0.1219868395786658],

[0.1349629938026717
 0.12319053943111329
 0.09696388576924422]'',

[-0.20382526255910166
 0.2038252625591016]'')

for i in range(1,length(outs))
    if outs[i] != truth[i]
        println("Incorrect! Outs:\n", outs[i], "\nvs\n", truth[i])
    end
end

true_r_h = [0.3344883118838543 0.37630407100318514 0.5673596773818216]'
if r.h != true_r_h
    println("Incorrect! r.h:\n", r.h, "\nvs\n", true_r_h)
end